# Practical Exam: House sales

RealAgents is a real estate company that focuses on selling houses.

RealAgents sells a variety of types of house in one metropolitan area.

Some houses sell slowly and sometimes require lowering the price in order to find a buyer.

In order to stay competitive, RealAgents would like to optimize the listing prices of the houses it is trying to sell.

They want to do this by predicting the sale price of a house given its characteristics.

If they can predict the sale price in advance, they can decrease the time to sale.


## Data

The dataset contains records of previous houses sold in the area.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton'. </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced" (two shared walls), "Semi-detached" (one shared wall), or "Detached" (no shared walls). </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |


# Task 1

The team at RealAgents knows that the city that a property is located in makes a difference to the sale price.

Unfortuntately they believe that this isn't always recorded in the data.

Calculate the number of missing values of the `city`.

 - You should use the data in the file "house_sales.csv".

 - Your output should be an object `missing_city`, that contains the number of missing values in this column.

In [24]:
# Use this cell to write your code for Task 1
# numeric imputation
for col in df.select_dtypes(include=[np.number]):
    df[col] = df[col].fillna(df[col].median())


# categorical imputation
for col in df.select_dtypes(exclude=[np.number]):
    df[col] = df[col].fillna('Unknown')

In [23]:
import pandas as pd
from io import StringIO

# Create DataFrame from the CSV data
df = pd.read_csv('/content/house_sales.csv')

# Count missing values in city column (represented as "--")
missing_city = (df['city'] == '--').sum()

print(f"Number of missing values in city column: {missing_city}")

Number of missing values in city column: 73


# Task 2

Before you fit any models, you will need to make sure the data is clean.

The table below shows what the data should look like.

Create a cleaned version of the dataframe.

 - You should start with the data in the file "house_sales.csv".

 - Your output should be a dataframe named `clean_data`.

 - All column names and values should match the table below.


| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton' </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced", "Semi-detached", or "Detached". </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |

In [19]:
import pandas as pd
import numpy as np

# Load the data
df = pd.read_csv('house_sales.csv')

# Create a copy to work with
clean_data = df.copy()

# 1. house_id - No changes needed (no missing values based on criteria)
print("Checking house_id missing values:", clean_data['house_id'].isnull().sum())

# 2. city - Replace missing values with "Unknown"
print("City missing values before:", clean_data['city'].isnull().sum())
clean_data['city'] = clean_data['city'].replace('--', 'Unknown')
clean_data['city'] = clean_data['city'].fillna('Unknown')
print("City missing values after:", clean_data['city'].isnull().sum())

# 3. sale_price - Remove missing entries
print("Sale price missing values before:", clean_data['sale_price'].isnull().sum())
clean_data = clean_data.dropna(subset=['sale_price'])
print("Sale price missing values after:", clean_data['sale_price'].isnull().sum())

# 4. sale_date - Replace missing values with 2023-01-01
print("Sale date missing values before:", clean_data['sale_date'].isnull().sum())
clean_data['sale_date'] = clean_data['sale_date'].fillna('2023-01-01')
print("Sale date missing values after:", clean_data['sale_date'].isnull().sum())

# 5. months_listed - Replace missing values with mean, rounded to one decimal place
print("Months listed missing values before:", clean_data['months_listed'].isnull().sum())
months_mean = clean_data['months_listed'].mean()
clean_data['months_listed'] = clean_data['months_listed'].fillna(round(months_mean, 1))
clean_data['months_listed'] = clean_data['months_listed'].round(1)
print("Months listed missing values after:", clean_data['months_listed'].isnull().sum())

# 6. bedrooms - Replace missing values with mean, rounded to nearest integer
print("Bedrooms missing values before:", clean_data['bedrooms'].isnull().sum())
bedrooms_mean = clean_data['bedrooms'].mean()
clean_data['bedrooms'] = clean_data['bedrooms'].fillna(round(bedrooms_mean))
clean_data['bedrooms'] = clean_data['bedrooms'].astype(int)
print("Bedrooms missing values after:", clean_data['bedrooms'].isnull().sum())

# 7. house_type - Replace missing values with most common house type
print("House type missing values before:", clean_data['house_type'].isnull().sum())
# Check for abbreviated values and standardize them
clean_data['house_type'] = clean_data['house_type'].replace({'Det.': 'Detached', 'Semi': 'Semi-detached', 'Terr.': 'Terraced'})
most_common_type = clean_data['house_type'].mode()[0]
clean_data['house_type'] = clean_data['house_type'].fillna(most_common_type)
print("House type missing values after:", clean_data['house_type'].isnull().sum())

# 8. area - Replace missing values with mean, rounded to one decimal place
print("Area missing values before:", clean_data['area'].isnull().sum())
# Extract numeric values from area column (remove 'sq.m.')
clean_data['area'] = clean_data['area'].str.replace(' sq.m.', '', regex=False).astype(float)
area_mean = clean_data['area'].mean()
clean_data['area'] = clean_data['area'].fillna(round(area_mean, 1))
clean_data['area'] = clean_data['area'].round(1)
print("Area missing values after:", clean_data['area'].isnull().sum())

# Verify the final cleaned data
print("\nFinal data shape:", clean_data.shape)
print("\nData types:")
print(clean_data.dtypes)
print("\nMissing values after cleaning:")
print(clean_data.isnull().sum())

clean_data

Checking house_id missing values: 0
City missing values before: 0
City missing values after: 0
Sale price missing values before: 0
Sale price missing values after: 0
Sale date missing values before: 0
Sale date missing values after: 0
Months listed missing values before: 31
Months listed missing values after: 0
Bedrooms missing values before: 0
Bedrooms missing values after: 0
House type missing values before: 0
House type missing values after: 0
Area missing values before: 0
Area missing values after: 0

Final data shape: (1500, 8)

Data types:
house_id           int64
city              object
sale_price         int64
sale_date         object
months_listed    float64
bedrooms           int64
house_type        object
area             float64
dtype: object

Missing values after cleaning:
house_id         0
city             0
sale_price       0
sale_date        0
months_listed    0
bedrooms         0
house_type       0
area             0
dtype: int64


,house_id,city,sale_price,sale_date,months_listed,bedrooms,house_type,area
0,1217792,Silvertown,55943,2021-09-12,5.4,2,Semi-detached,107.8
1,1900913,Silvertown,384677,2021-01-17,6.3,5,Detached,498.8
2,1174927,Riverford,281707,2021-11-10,6.9,6,Detached,542.5
3,1773666,Silvertown,373251,2020-04-13,6.1,6,Detached,528.4
4,1258487,Silvertown,328885,2020-09-24,8.7,5,Detached,477.1
...,...,...,...,...,...,...,...,...
1495,1123892,Riverford,198661,2022-02-17,7.3,5,Detached,432.2
1496,1327295,Poppleton,358304,2020-10-10,6.2,6,Detached,599.8
1497,1058161,Riverford,176612,2022-11-01,6.8,4,Detached,359.1
1498,1822037,Teasdale,197827,2021-04-03,5.4,3,Detached,253.7


# Task 3

The team at RealAgents have told you that they have always believed that the number of bedrooms is the biggest driver of house price.

Producing a table showing the difference in the average sale price by number of bedrooms along with the variance to investigate this question for the team.

 - You should start with the data in the file 'house_sales.csv'.

 - Your output should be a data frame named `price_by_rooms`.

 - It should include the three columns `bedrooms`, `avg_price`, `var_price`.

 - Your answers should be rounded to 1 decimal place.   

In [18]:
# Use this cell to write your code for Task 3
import pandas as pd
import numpy as np

# Load the data
df = pd.read_csv('/content/house_sales.csv')

# Clean the data - remove rows with missing values in bedrooms or sale_price
df = df.dropna(subset=['bedrooms', 'sale_price'])

# Calculate average price and variance by number of bedrooms
price_by_rooms = df.groupby('bedrooms')['sale_price'].agg(['mean', 'var']).reset_index()
price_by_rooms.columns = ['bedrooms', 'avg_price', 'var_price']

# Round to 1 decimal place
price_by_rooms['avg_price'] = price_by_rooms['avg_price'].round(1)
price_by_rooms['var_price'] = price_by_rooms['var_price'].round(1)

price_by_rooms

,bedrooms,avg_price,var_price
0,2,67076.4,5.652896e+08
1,3,154665.1,2.378289e+09
2,4,234704.6,1.725211e+09
3,5,301515.9,2.484328e+09
4,6,375741.3,3.924432e+09


# Task 4

Fit a baseline model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `base_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [15]:
# Use this cell to write your code for Task 4
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Load the training data
train_data = pd.read_csv('/content/train.csv')

# Explore the data structure
print("Data columns:", train_data.columns.tolist())
print("Data types:\n", train_data.dtypes)
print("Missing values:\n", train_data.isnull().sum())

# Prepare the data for modeling
# Encode categorical variables
le_city = LabelEncoder()
le_house_type = LabelEncoder()

train_data['city_encoded'] = le_city.fit_transform(train_data['city'])
train_data['house_type_encoded'] = le_house_type.fit_transform(train_data['house_type'])

# Select features for the baseline model
# Using basic features that are likely to correlate with price
features = ['bedrooms', 'area', 'city_encoded', 'house_type_encoded', 'months_listed']

X_train = train_data[features]
y_train = train_data['sale_price']

# Fit a simple linear regression model as baseline
baseline_model = LinearRegression()
baseline_model.fit(X_train, y_train)

# Load validation data
validation_data = pd.read_csv('/content/validation.csv')

# Preprocess validation data the same way
validation_data['city_encoded'] = le_city.transform(validation_data['city'])
validation_data['house_type_encoded'] = le_house_type.transform(validation_data['house_type'])

X_val = validation_data[features]

# Make predictions
predictions = baseline_model.predict(X_val)

# Create the result dataframe
base_result = pd.DataFrame({
    'house_id': validation_data['house_id'],
    'price': predictions
})

# Ensure price values are reasonable (non-negative)
base_result['price'] = base_result['price'].clip(lower=0)

print("Baseline model predictions completed")
print("Sample predictions:\n", base_result.head())
print("Prediction statistics:")
print("Min price:", base_result['price'].min())
print("Max price:", base_result['price'].max())
print("Mean price:", base_result['price'].mean())

Data columns: ['house_id', 'city', 'sale_price', 'sale_date', 'months_listed', 'bedrooms', 'house_type', 'area']
Data types:
 house_id           int64
city              object
sale_price         int64
sale_date         object
months_listed    float64
bedrooms           int64
house_type        object
area             float64
dtype: object
Missing values:
 house_id         0
city             0
sale_price       0
sale_date        0
months_listed    0
bedrooms         0
house_type       0
area             0
dtype: int64
Baseline model predictions completed
Sample predictions:
    house_id          price
0   1331375  114410.640068
1   1630115  298257.275635
2   1645745  385953.871714
3   1336775  122798.036772
4   1888274  272134.780558
Prediction statistics:
Min price: 5040.8993559880255
Max price: 434181.2019089885
Mean price: 233472.25182989647


# Task 5

Fit a comparison model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `compare_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [17]:
# Use this cell to write your code for Task 5
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings('ignore')

# Load the training data
train_data = pd.read_csv('/content/train.csv')

# Feature engineering for the comparison model
def engineer_features(df):
    df = df.copy()

    # Extract year and month from sale_date
    df['sale_date'] = pd.to_datetime(df['sale_date'])
    df['sale_year'] = df['sale_date'].dt.year
    df['sale_month'] = df['sale_date'].dt.month
    df['sale_quarter'] = df['sale_date'].dt.quarter

    # Create area per bedroom feature
    df['area_per_bedroom'] = df['area'] / df['bedrooms']
    # Replace infinite values with the mean of the area column
    df['area_per_bedroom'] = df['area_per_bedroom'].replace([np.inf, -np.inf], df['area'].mean())


    # Create binary features for house types
    df['is_detached'] = (df['house_type'] == 'Detached').astype(int)
    df['is_semi_detached'] = (df['house_type'] == 'Semi-detached').astype(int)
    df['is_terraced'] = (df['house_type'] == 'Terraced').astype(int)

    # Create interaction features
    df['bedrooms_area_interaction'] = df['bedrooms'] * df['area']

    # Price per square meter (target encoding inspiration)
    df['price_per_sqm'] = df['sale_price'] / df['area']

    return df

# Engineer features for training data
train_data_engineered = engineer_features(train_data)

# Encode categorical variables with more sophisticated approach
le_city = LabelEncoder()
train_data_engineered['city_encoded'] = le_city.fit_transform(train_data_engineered['city'])

# Calculate average price per city for target encoding
city_avg_price = train_data_engineered.groupby('city')['sale_price'].mean().to_dict()
train_data_engineered['city_avg_price'] = train_data_engineered['city'].map(city_avg_price)

# Calculate average price per house type
house_type_avg_price = train_data_engineered.groupby('house_type')['sale_price'].mean().to_dict()
train_data_engineered['house_type_avg_price'] = train_data_engineered['house_type'].map(house_type_avg_price)

# Features for the comparison model
comparison_features = [
    'bedrooms', 'area', 'months_listed', 'city_encoded',
    'sale_year', 'sale_month', 'area_per_bedroom',
    'is_detached', 'is_semi_detached', 'is_terraced',
    'bedrooms_area_interaction', 'city_avg_price', 'house_type_avg_price'
]

X_train_compare = train_data_engineered[comparison_features]
y_train_compare = train_data_engineered['sale_price']

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_compare)

# Use Random Forest as comparison model (more powerful than linear regression)
compare_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=15,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1
)

# Fit the comparison model
compare_model.fit(X_train_scaled, y_train_compare)

# Load and preprocess validation data
validation_data = pd.read_csv('/content/validation.csv')

def preprocess_validation_data(df, le_city, city_avg_price, house_type_avg_price):
    df = df.copy()

    # Extract year and month from sale_date
    df['sale_date'] = pd.to_datetime(df['sale_date'])
    df['sale_year'] = df['sale_date'].dt.year
    df['sale_month'] = df['sale_date'].dt.month
    df['sale_quarter'] = df['sale_date'].dt.quarter

    # Create area per bedroom feature
    df['area_per_bedroom'] = df['area'] / df['bedrooms']
    # Replace infinite values with the mean of the area column
    df['area_per_bedroom'] = df['area_per_bedroom'].replace([np.inf, -np.inf], df['area'].mean())

    # Create binary features for house types
    df['is_detached'] = (df['house_type'] == 'Detached').astype(int)
    df['is_semi_detached'] = (df['house_type'] == 'Semi-detached').astype(int)
    df['is_terraced'] = (df['house_type'] == 'Terraced').astype(int)

    # Create interaction features
    df['bedrooms_area_interaction'] = df['bedrooms'] * df['area']

    # Encode city
    df['city_encoded'] = le_city.transform(df['city'])

    # Add target encoding features
    df['city_avg_price'] = df['city'].map(city_avg_price)
    df['house_type_avg_price'] = df['house_type'].map(house_type_avg_price)


    return df

# Preprocess validation data
validation_processed = preprocess_validation_data(
    validation_data, le_city, city_avg_price, house_type_avg_price
)

X_val_compare = validation_processed[comparison_features]

# Scale validation features
X_val_scaled = scaler.transform(X_val_compare)

# Make predictions with comparison model
compare_predictions = compare_model.predict(X_val_scaled)

# Create the result dataframe
compare_result = pd.DataFrame({
    'house_id': validation_data['house_id'],
    'price': compare_predictions
})

# Ensure price values are reasonable (non-negative)
compare_result['price'] = compare_result['price'].clip(lower=0)

print("Comparison model predictions completed")
print("Sample predictions:\n", compare_result.head())
print("Prediction statistics:")
print("Min price:", compare_result['price'].min())
print("Max price:", compare_result['price'].max())
print("Mean price:", compare_result['price'].mean())
print("Median price:", compare_result['price'].median())

# Show feature importance for insight
feature_importance = pd.DataFrame({
    'feature': comparison_features,
    'importance': compare_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nFeature importance:")
print(feature_importance)

Comparison model predictions completed
Sample predictions:
    house_id          price
0   1331375   80945.413161
1   1630115  309315.196603
2   1645745  402566.347831
3   1336775  107090.540119
4   1888274  266021.899444
Prediction statistics:
Min price: 35541.43094444444
Max price: 478452.60867279937
Mean price: 233874.14598056176
Median price: 240501.59520833334

Feature importance:
                      feature  importance
10  bedrooms_area_interaction    0.463394
1                        area    0.357077
7                 is_detached    0.048600
3                city_encoded    0.043636
12       house_type_avg_price    0.042355
11             city_avg_price    0.033893
6            area_per_bedroom    0.003871
2               months_listed    0.002583
5                  sale_month    0.001503
8            is_semi_detached    0.001226
9                 is_terraced    0.001041
4                   sale_year    0.000725
0                    bedrooms    0.000097
